In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
def swish(x):
    return x*tf.nn.sigmoid(x)

In [15]:
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.convnext import ConvNeXtBase

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

In [5]:
x_train=tf.keras.applications.convnext.preprocess_input(x_train)
x_test=tf.keras.applications.convnext.preprocess_input(x_test)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model_base=ConvNeXtBase(
    include_top=False,
    weights='imagenet',
    input_shape=(64,64,3)
)

350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [9]:
base_model_base.trainable=False

In [10]:
model=models.Sequential([
    base_model_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ convnext_base (Functional)           │ (None, 2, 2, 1024)          │      87,566,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 89,305,060 (340.67 MB)

 Trainable params: 1,735,012 (6.62 MB)

 Non-trainable params: 87,570,048 (334.05 MB)

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [12]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [16]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [17]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [18]:
import time
start_time=time.time()

In [20]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1172/1172 ━━━━━━━━━━━━━━━━━━━━ 95s 61ms/step - accuracy: 0.0993 - loss: 4.5681 - val_accuracy: 0.5630 - val_loss: 1.8576
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 47ms/step - accuracy: 0.3496 - loss: 2.8003 - val_accuracy: 0.6184 - val_loss: 1.4941
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 59s 50ms/step - accuracy: 0.4267 - loss: 2.3402 - val_accuracy: 0.6458 - val_loss: 1.3453
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 58s 49ms/step - accuracy: 0.4580 - loss: 2.1298 - val_accuracy: 0.6622 - val_loss: 1.2623
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 48ms/step - accuracy: 0.4911 - loss: 1.9770 - val_accuracy: 0.6720 - val_loss: 1.2052
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 48ms/step - accuracy: 0.5060 - loss: 1.8869 - val_accuracy: 0.6795 - val_loss: 1.1697
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 48ms/step - accuracy: 0.5281 - loss: 1.7841 - val_accuracy: 0.6882 - val_loss: 1.1343
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 48ms/step - accuracy: 0.5363 - loss: 1.74

In [21]:
head_training_time=time.time()-start_time

In [22]:
base_model_base.trainable=True

In [23]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [24]:
start_fine_tune=time.time()

In [25]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 217s 124ms/step - accuracy: 0.6989 - loss: 1.0665 - val_accuracy: 0.8285 - val_loss: 0.5869
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 109s 93ms/step - accuracy: 0.7893 - loss: 0.7358 - val_accuracy: 0.8500 - val_loss: 0.5147
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 109s 93ms/step - accuracy: 0.8222 - loss: 0.6369 - val_accuracy: 0.8572 - val_loss: 0.4878
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 109s 93ms/step - accuracy: 0.8484 - loss: 0.5358 - val_accuracy: 0.8634 - val_loss: 0.4721
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 109s 93ms/step - accuracy: 0.8610 - loss: 0.4809 - val_accuracy: 0.8686 - val_loss: 0.4493
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 109s 92ms/step - accuracy: 0.8758 - loss: 0.4254 - val_accuracy: 0.8740 - val_loss: 0.4405
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 109s 92ms/step - accuracy: 0.8888 - loss: 0.3885 - val_accuracy: 0.8746 - val_loss: 0.4407
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 108s 92ms/step - accuracy:

In [26]:
fine_tune_time=time.time()-start_fine_tune

In [27]:
test_loss, test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")


Test Accuracy: 87.77%
Total training time: 2371.96 seconds


In [28]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8802 - loss: 0.4292
Validation accuracy: 87.8560%


In [29]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 34ms/step


In [30]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for ConvNext Base:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for ConvNext Base:
Accuracy: 0.8777
Precision: 0.8800
Recall: 0.8777
F1-Score: 0.8779


In [31]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["ConvNext Base"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [32]:
print(results)

           Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  ConvNext Base         0.8777              0.87856   0.445463   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             1174.192009           1197.766773      89305060  


In [33]:
results.to_csv("ConvNext Base.csv",index=False)
print("\nResults saved to ConvNext Base.csv")


Results saved to ConvNext Base.csv
